Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>DESIRED_ACCURACY):
          print("\nReached 99.9% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

--2020-12-06 23:55:38--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 74.125.20.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2020-12-06 23:55:39 (103 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [11]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
  keras.layers.Conv2D(64, (3,3), input_shape=(150, 150, 3), activation="relu"),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(128, (3,3), activation="relu"),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(128, (3,3), activation="relu"),
  keras.layers.MaxPooling2D(2),
  keras.layers.Flatten(),
  keras.layers.Dense(40, activation="relu"),
  keras.layers.Dense(1, activation="sigmoid")
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001), 
              loss="binary_crossentropy", metrics="acc")

In [13]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/tmp/h-or-s', 
  target_size=(150, 150), batch_size=10, class_mode="binary")

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [14]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit_generator(train_generator, steps_per_epoch=8, epochs=20, callbacks=[callbacks], verbose=1)
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/20
8/8 [==============================] - 6s 768ms/step - loss: 1.0136 - acc: 0.5500
Epoch 2/20
8/8 [==============================] - 6s 770ms/step - loss: 0.6374 - acc: 0.7125
Epoch 3/20
8/8 [==============================] - 6s 765ms/step - loss: 0.6804 - acc: 0.8250
Epoch 4/20
8/8 [==============================] - 6s 761ms/step - loss: 0.1902 - acc: 0.9500
Epoch 5/20
8/8 [==============================] - 6s 764ms/step - loss: 0.0983 - acc: 0.9625
Epoch 6/20
8/8 [==============================] - 6s 762ms/step - loss: 0.1438 - acc: 0.9375
Epoch 7/20
8/8 [==============================] - 6s 762ms/step - loss: 0.0820 - acc: 0.9625
Epoch 8/20
8/8 [==============================] - ETA: 0s - loss: 0.0174 - acc: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 6s 766ms/step - loss: 0.0174 - acc: 1.0000
